# Notebook 5: Looking for Klein bottles

Inspired by the [paper](https://link.springer.com/article/10.1007/s11263-007-0056-x) by Carlsson et al., we are going to build part of the pipeline for finding the topology of 3x3 patches of pixels from images. 

_Spoilers_ We probably won't find the Klein bottle, since our analysis differs from that in the paper in a number of ways:
- We are using one image and will eventually have about 300 patches to work with. Carlsson et al. start with 4 million patches from thousands of images and sample down to 10,000 at the last step.
- Carlsson et al. deliberately throw in certain patches from non-dense regions using their set $Q$. We will not be doing that here.

Nonetheless, I hope that this exercise gives some insight into what goes into this kind of analysis and also introduces you to `ripser`, which is far more efficient than `gudhi` for VR complexes.

In [ ]:
!pip install numpy==1.22.3

In [ ]:
!pip install ripser

In [ ]:
!pip install persim

In [ ]:
from PIL import Image
from PIL import ImageOps
import matplotlib.pyplot as plt
import random
from ripser import ripser
from persim import plot_diagrams
import numpy as np

## Generating the 3x3 patch data

In their paper, Carlsson et al. use a large database of images and eventually sample down before computing persistent homology. We are going to do everything to a single image, so we should not expect the same kind of results. However, we will at least grapple with what is required for this kind of analysis. The image I chose might be recognizable to you.

In [ ]:
#load image
image = Image.open("./clocktower.jpeg")
#make grayscale
gray = ImageOps.grayscale(image)
array = np.array(gray)

In [ ]:
plt.imshow(array, cmap='gray')

We want to extract a lot of 3x3 patches and do some processing to them (namely take the log of each pixel value and subtract the mean). To make the code cleaner, I've wrapped that all up into a function below. Make sure you understand what this function is doing.

In [ ]:
def get3x3(x,y,array):
    patch = np.array(
        array[x:x+3, y:y+3],
        dtype=float
    ).flatten()
    patch = np.log(1+patch)
    patch = patch - np.mean(patch)
    return patch

To collect our patches, we'll choose 5000 random coordinate pairs and collect the 3x3 patches at those coordinates.

In [ ]:
random.seed(2022) #this is so that it does the same thing every time, but you can remove it
sample_x_coordinates = np.random.choice(range(array.shape[0]-3), 10000, replace=True)
sample_y_coordinates = np.random.choice(range(array.shape[1]-3), 10000, replace=True)

In [ ]:
patches = []
for x, y in zip(sample_x_coordinates, sample_y_coordinates):
    patches.append(get3x3(x,y,array))

## Pick high contrast patches

Each 3x3 patch has been flattened into a vector of length 9 by our function. We are now going to select the top 20% highest contrast patches from among the 5000. The contrast of a patch $\vec{v}$ is defined in the paper as $\vec{v}^{\mathrm{T}} D \vec{v}$ for a particular matrix $D$. 

In [ ]:
D = np.array([
    [2,-1,0,-1,0,0,0,0,0],
    [-1,3,-1,0,-1,0,0,0,0],
    [0,-1,2,0,0,-1,0,0,0],
    [-1,0,0,3,-1,0,-1,0,0],
    [0,-1,0,-1,4,-1,0,-1,0],
    [0,0,-1,0,-1,3,0,0,-1],
    [0,0,0,-1,0,0,2,-1,0],
    [0,0,0,0,-1,0,-1,3,-1],
    [0,0,0,0,0,-1,0,-1,2]
])

Let's order our patches by their contrast values. To do this we're going to use the Python function `sorted` in a particular way. The function `sorted` allows you to specify the key by which to sort the values you hand it. For example, you might want to sort words by length not by alphabetical order. Let's make a function that returns the contrast value first.

In [ ]:
def Dnorm(patch):
    return np.sqrt(patch.T @ D @ patch)

Now we call `sorted` with `key=Dnorm` and also `reverse=True` so that we get it in descending order. After that we can take the top 1000 patches by constrast value.

In [ ]:
patches_ordered_by_contrast = sorted(
    patches,
    key = lambda x: Dnorm(x),
    reverse=True
)

In [ ]:
top_contrast_patches = patches_ordered_by_contrast[:2000]

Let's look at the highest contrast patch just to get an idea. (We'll have to reshape it back to a 3x3 array to do so).

In [ ]:
plt.imshow(top_contrast_patches[0].reshape(3,3), cmap='gray')

## A change of coordinates

We are now equipped with 1000 vectors of length 9. Carlsson et al. (following earlier [work](https://dash.harvard.edu/bitstream/handle/1/3637108/mumford_nonlinstatpatches.pdf?sequence=1) of Lee, Pederson and Mumford) normalize these vectors to have contrast value 1 and then apply a change of basis so that the vectors are arranged on the surface of the unit ball in $\mathbb{R}^8$. In other words they do linear algebra.

In [ ]:
#normalize by contrast
normalized_patches = [
    v/Dnorm(v) for v in top_contrast_patches
]

In [ ]:
#this is our basis we're going to use
e1 = 1/np.sqrt(6)*np.array([1,0,-1,1,0,-1,1,0,-1])
e2 = 1/np.sqrt(6)*np.array([1,1,1,0,0,0,-1,-1,-1])
e3 = 1/np.sqrt(54)*np.array([1,-2,1,1,-2,1,1,-2,1])
e4 = 1/np.sqrt(54)*np.array([1,1,1,-2,-2,-2,1,1,1])
e5 = 1/np.sqrt(8)*np.array([1,0,-1,0,0,0,-1,0,1])
e6 = 1/np.sqrt(48)*np.array([1,0,-1,-2,0,2,1,0,-1])
e7 = 1/np.sqrt(48)*np.array([1,-2,1,0,0,0,-1,2,-1])
e8 = 1/np.sqrt(216)*np.array([1,-2,1,-2,4,-2,1,-2,1])

In [ ]:
#we build a matrix that changes basis (and dimension)
gamma = np.diag([
    1/np.linalg.norm(e)**2 for e in [e1,e2,e3,e4,e5,e6,e7,e8]
])
A = np.array([e1,e2,e3,e4,e5,e6,e7,e8]).T

In [ ]:
#it's now time to apply the change of basis
patches_in_R8 = [gamma@A.T@v for v in normalized_patches]

It's okay if you didn't follow everything above, but let's at least see the end result. Our new vectors should all have length 8, so let's check that.

In [ ]:
len(patches_in_R8[0])

They should also have length $1$.

In [ ]:
np.linalg.norm(patches_in_R8[0])

This means our vectors lie on the unit sphere in $\mathbb{R}^8$, as required. 

## Density sampling
We're still not done preprocessing. We want to take points only from the densest regions of the point cloud in $\mathbb{R}^8$ that we can find. To do this, we pick a $k$ and compute the distance from each point to its $k^{th}$ nearest neighbor. Can you see how this value is low in dense regions and high in sparse regions? As before, we define a function which tells us the distance to the $k^{th}$ nearest neighbor and use it as a key to sort our vectors. It'll be a bit more efficient if we precompute all distances between all pairs of vectors*.

_*this is not the most computationally efficient thing to do but it's the simplest_

In [ ]:
def distance_to_knn(vector):
    distances_to_all_vectors = [np.linalg.norm(vector-v) for v in patches_in_R8]
    return sorted(distances_to_all_vectors)[k]

In [ ]:
k = 15
patches_sorted_by_density = sorted(
    patches_in_R8,
    key = distance_to_knn
)

In [ ]:
patches_from_high_density = patches_sorted_by_density[:300]

We finally have the set of vectors we want to run persistent homology on. As a last step, let's organize our list of vectors into a single array like this.

In [ ]:
data = np.vstack(patches_from_high_density)

## Exercise 1: VR persistence using `ripser`

Let's compute the VR persistence diagram for `data` now. For computing VR persistence on large point clouds, I recommend against `gudhi`. So for this exercise, compute and plot the 0 and 1 dimensional persistence (on one plot) of `data` using `ripser` following [this](https://ripser.scikit-tda.org/en/latest/notebooks/Basic%20Usage.html) simple example.

If you do it right, you should see one clear $H_1$ point. This is in line with what [previous studies](https://diglib.eg.org/bitstream/handle/10.2312/SPBG.SPBG04.157-166/157-166.pdf?sequence=1&isAllowed=y) found with a small sample size and relatively low value of $k$.

## Exercise 2: $H_2$ homology
What about $H_2$? Run `ripser` again with the keyword `maxdim=2` and see what happens. How much longer did it take to run `ripser`? When I run it I don't get any $H_2$ points away from the diagonal, which shows that our sample size is not large enough to see the Klein bottle structure.

## Exercise 3: Removing the density sampling

Let's see whether the density sampling mattered. Repeat Exercise 1 but instead of using the vectors in `data`, use the vectors in `patches_in_R8` directly. What changes?

## Challenge questions

- Choose your own image and repeat the above analysis. Might I suggest your favorite album cover?
- Adjust the number of patches sampled (5000), the number of high contrast patches taken (1000), the value of $k$ (15) and/or the number of high density patches taken (300) and see if you see anything interesting. 